# WHY CONVOLUTIONS WORK

In [ ]:
# IMPORTS
import numpy as np
import scipy as sp
import skimage.io as io
import skimage.transform as t
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Some Samples
Let us now create some samples to work with. We shall attempt to use convolutions to distinguish between a square and a rhombus.

In [ ]:
# CREATING SOME SAMPLES TO WORK WITH

square = np.zeros((256,256))
square[64:192,64:192]=1
io.imsave('sample_data/square.png',(square*255).astype(np.uint8))
rhombus = t.rotate(square,45)
io.imsave('sample_data/rhombus.png',(rhombus*255).astype(np.uint8))
plt.rcParams['figure.dpi']= 75
io.imshow_collection([square,rhombus])

### Now let us create a custom kernel

In [ ]:
# Desinging a kernel
kernel_d = np.array ([[-1,-1, 0],
                      [-1, 0, 1],
                      [ 0, 1, 1]]).astype(np.int8)
kernel_h = np.array ([[-1,-1,-1],
                      [ 0, 0, 0],
                      [ 1, 1, 1]]).astype(np.int8)
plt.rcParams['figure.dpi']= 75
io.imshow_collection([kernel_h,kernel_d])

In [ ]:
# Defining convolution for single kernel 
def convolute (image, kernel, bias):
    H,W = image.shape
    kH,kW = kernel.shape
    H_out = H-kH+1                                               # output height
    W_out = W-kW+1                                               # output width
    output = np.zeros((H_out,W_out))                             # Initializing output
    for x in range (H_out):                                      # sliding kernel vertically
        for y in range (W_out):                                  # sliding kernel horizontally
            ix,iy = x+int(kW/2),y+int(kH/2)                      # centre of the kernel on the image
            region = image[ix-int(kW/2):ix+int(kW/2)+1,
                           iy-int(kH/2):iy+int(kH/2)+1]          # region of image corresponding to kernel position
            output[x,y] = np.multiply(kernel,region).sum()+bias  # Affine transformation: y = sum(w*x)+b
            output[x,y] = output[x,y] if output[x,y]>=0 else 0   # Non-linearity (relu)
    if (output.max(_map) != 0):
        output = output/output.max()
    return (output)

image, kernel = rhombus, kernel_d
op = convolute(image,kernel,-2)
plt.rcParams['figure.dpi']= 75
io.imshow(op)

### Now more kernels

In [113]:
# More Kernels
kernels = np.zeros((8,3,3))
kernels[0] = np.array ([[ 1, 1, 1],[ 0, 0, 0],[-1,-1,-1]])    # South
kernels[1] = np.array ([[-1,-1,-1],[ 0, 0, 0],[ 1, 1, 1]])    # North
kernels[2] = np.array ([[ 1, 0,-1],[ 1, 0,-1],[ 1, 0,-1]])    # East
kernels[3] = np.array ([[-1, 0, 1],[-1, 0, 1],[-1, 0, 1]])    # West
kernels[4] = np.array ([[ 1, 1, 0],[ 1, 0,-1],[ 0,-1,-1]])    # Southeast
kernels[5] = np.array ([[ 0,-1,-1],[ 1, 0,-1],[ 1, 1, 0]])    # Northeast
kernels[6] = np.array ([[ 0, 1, 1],[-1, 0, 1],[-1,-1, 0]])    # Southwest
kernels[7] = np.array ([[-1,-1, 0],[-1, 0, 1],[ 0, 1, 1]])    # Northwest
biases = [-2]*8

In [114]:
# Redefining convolution for multiple kernels
def convolute (image, kernels, biases):
    H,W = image.shape
    kC,kH,kW = kernels.shape                                               # Multiple Kernels (kC)
    H_out = H-kH+1                                                         
    W_out = W-kW+1                                                         
    output = np.zeros((H_out,W_out,kC))                                    # Initializing output for multi kernels
    for k in range (kC):                                                   # Repeating for multiple kernels *
        kernel = kernels[k]
        for x in range (H_out):                                            
            for y in range (W_out):                                        
                ix,iy = x+int(kW/2),y+int(kH/2)                           
                region = image[ix-int(kW/2):ix+int(kW/2)+1,
                               iy-int(kH/2):iy+int(kH/2)+1]                
                output[x,y,k] = np.multiply(kernel,region).sum()+biases[k] # Affine transformation for every kernel
                output[x,y,k] = output[x,y,k] if output[x,y,k]>=0 else 0   # Non-linearity (relu) for every kernel
    if (output.max() != 0):
        output = output/output.max()
    return (output)
image = square
op = convolute(image,kernels,biases)
for i in range (op.shape[2]):
    io.imsave('sample_data/kernel_output%d.png'%i,(op[:,:,i]))


### Doing convolutions on multi-channeled inputs

In [173]:
# Redefining convolution for multi_channeled inputs

def convolute (image, kernels, biases):
    H,W,C = image.shape                         # Images must have 3 dims (Height, Width, Channels)
    k_OC,kH,kW,k_IC = kernels.shape             # Kernel must have 4 dims (Height, Width, I/P channels, O/P channels)                                
    H_out = H-kH+1                                                         
    W_out = W-kW+1                                                         
    output = np.zeros((H_out,W_out,k_OC))                                    
    for k in range (k_OC):                                                   
        kernel = kernels[k]
        for x in range (H_out):                                            
            for y in range (W_out):                                        
                ix,iy = x+int(kW/2),y+int(kH/2)                           
                region = image[ix-int(kW/2):ix+int(kW/2)+1,
                               iy-int(kH/2):iy+int(kH/2)+1]                
                output[x,y,k] = np.multiply(kernel,region).sum()+biases[k] 
                output[x,y,k] = output[x,y,k] if output[x,y,k]>=0 else 0   
    if (output.max() != 0):
        output = output/output.max()
    return (output)

# But we have to reshape inputs and kernels to match required dimensions
image = rhombus
image = image.reshape(image.shape[0],image.shape[1],1)
l1_kernels = kernels.reshape(kernels.shape[0],kernels.shape[1],kernels.shape[2],1)
biases = [-2]*8
l1_out = convolute(image,l1_kernels,biases)
for i in range (l1_out.shape[2]):
    io.imsave('sample_data/l1_output%d.png'%i,(l1_out[:,:,i]))
    


### Mixing features with higher layers

In [174]:
l2_kernels = np.array([[1,1,1,1,0,0,0,0],[0,0,0,0,1,1,1,1]]).astype(np.int8)
l2_kernels = l2_kernels.reshape(l2_kernels.shape[0],1,1,l2_kernels.shape[1])
print(l2_kernels.shape)

(2, 1, 1, 8)


In [175]:
l2_out = convolute(l1_out,l2_kernels,[0,0])
for i in range (l2_out.shape[2]):
    io.imsave('sample_data/l2_output%d.png'%i,(l2_out[:,:,i]))

### Classifying outputs using features

In [176]:
sq_kernel = np.array([[[1,0]]*254]*254)
rh_kernel = np.array([[[0,1]]*254]*254)
output_kernels = np.array([sq_kernel,rh_kernel]).astype(np.uint8)

In [177]:
outputs = convolute(l2_out,output_kernels,[-5,-5])
if (outputs.squeeze()[0]==1):
    print ("It is a square")
if (outputs.squeeze()[1]==1):
    print("It is a rhombus")

It is a rhombus
